# An example of using EASYMORE to extract data for point data, in this case capitals of the world
## Source netCDF file are ERA5 limited to North American Domain and Caribbean domain
## In this example, we initialize easymore using a configuration dictionary

In [1]:
# loading EASYMORE
from easymore.easymore import easymore
import json #import decoder

# initializing EASYMORE object
esmr = easymore()

json_str   = '''
{
"case_name"   : "ERA5_capitals_north_america",
"temp_dir"    : "./temporary/",
"target_shp"  : "./data/target_shapefiles/Capitals_point.shp",
"source_nc"   : "./data/Source_nc_ERA5/ERA5_NA_*.nc",
"var_names"   : ["airtemp"],
"var_lon"     : "longitude",
"var_lat"     : "latitude",
"var_time"    : "time",
"output_dir"  : "./output/"
}
'''

json_obj = json.loads(json_str) #???
esmr = esmr.from_json(json_obj) #???

# decoder.JSONDecoder

# execute EASYMORE
esmr.nc_remapper()

EASYMORE version 1.1.1 is initiated.
{'case_name': 'ERA5_capitals_north_america', 'temp_dir': './temporary/', 'target_shp': './data/target_shapefiles/Capitals_point.shp', 'source_nc': './data/Source_nc_ERA5/ERA5_NA_*.nc', 'var_names': ['airtemp'], 'var_lon': 'longitude', 'var_lat': 'latitude', 'var_time': 'time', 'output_dir': './output/'}


TypeError: the JSON object must be str, bytes or bytearray, not dict

--------------
--------------
# Visualization of the source and the results

In [ ]:
# visualize the result
import geopandas as gpd
import xarray as xr
import glob
import matplotlib.pyplot as plt
import matplotlib
font = {'family' : 'Times New Roman',
         'weight' : 'bold',
         'size'   : 24}
matplotlib.rc('font', **font)

shp = gpd.read_file( esmr.temp_dir + esmr.case_name + '_intersected_shapefile.shp')
#print(shp)

# the intersected shapefile shows that the ID_t of Washington, D.C., Ottawa and Mexico City is 187, 102 and 192 meaning that
# they corresponse to the ID of 187 and 192 in the original shapefile of the capitals of the world
# many of the capitals are not in the intersection as the ERA5 data is limited to North America and Caribbean


# merge the output file into one xarray dataset
ds = xr.merge([xr.open_dataset(f) for f in sorted(glob.glob(esmr.output_dir+esmr.case_name +'*.nc'))])
#print(ds)
ds.airtemp [:,:] = ds.airtemp[:,:] - 273; # Kelvin to Celsius

# extract the data and plotting
fig, axes = plt.subplots(1,1,figsize=(15,7), facecolor='white')

WashingtonDC = ds.sel(ID=187, method = 'nearest') # ID_t, the target ID for Washington DC is 187
WashingtonDC.airtemp.plot(ax = axes, label='Washington DC')

MexicoCity = ds.sel(ID=192, method = 'nearest') # ID_t, the target ID for Mexico City is 192
MexicoCity.airtemp.plot(ax = axes, label='Mexico City')

Ottawa = ds.sel(ID=102, method = 'nearest') # ID_t, the target ID for Ottawa DC is 102
Ottawa.airtemp.plot(ax = axes, label='Ottawa')

plt.grid()
plt.ylabel ('Temprature [C]')
plt.title ('Temprature, Washington DC/Mexico City/Ottawa \n First three days of January 1979')
plt.legend()
plt.tight_layout()
plt.savefig('./fig/ERA5_North_America_Capitals.png')
